## Introduction
In this section, identifier (IDs) mapping is performed on an example metabolomics data set, which was original annotated using HMDB symbols.
The dataset has been preprocessed already, for details see step 7 and 8 of the multi-omics workflow at: https://github.com/BiGCAT-UM/Transcriptomics_Metabolomics_Analysis/tree/master/metabolomics_analysis .
We map the HGNCMDB symbols to ChEBI IDs, since tools downstream of this step require different input formats for the IDs.

We use one tool for this mapping: BridgeDb [doi:10.18129/B9.bioc.BridgeDbR].

## Setup


In [ ]:
# empty the R environment
rm (list = ls())
# check if libraries are already installed, otherwise install it
if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
if(!"rstudioapi" %in% installed.packages()) BiocManager::install("rstudioapi")
if(!"dplyr" %in% installed.packages()) BiocManager::install("dplyr")
if(!"BridgeDbR" %in% installed.packages()) BiocManager::install("BridgeDbR")
if(!"data.table" %in% installed.packages())install.packages("data.table")
if(!"knitr" %in% installed.packages())install.packages("knitr")
if(!"rJava" %in% installed.packages()) install.packages("rJava")
## See https://www.r-bloggers.com/2018/02/installing-rjava-on-ubuntu/ if you have issues with this package on Ubuntu.
if(!"ggplot2" %in% installed.packages()) install.packages("ggplot2")

#load libraries
suppressPackageStartupMessages({
  library(rstudioapi)
  library(dplyr)
  library(BridgeDbR)
  library(data.table)
  library(knitr)
  # library(rJava)
  library(ggplot2)

})

# set your working environment to the location where your current source file is saved into.
setwd(dirname(rstudioapi::getSourceEditorContext()$path))


## Importing dataset
The data will be read for two diseases


In [ ]:
mbx_dataset_CD <- read.csv("data/mbxData_CD.csv") %>% 
  select (HMDB_ID, Compound_Name, foldchange_disorder, p_values_disorder) %>% #filter out unused columns
  rename (HMDBID = HMDB_ID, log2FC = foldchange_disorder, pvalue = p_values_disorder) #change column names
mbx_dataset_UC <- read.csv("data/mbxData_UC.csv") %>%
  select (HMDB_ID, Compound_Name, foldchange_disorder, p_values_disorder) %>% #filter out unused columns
  rename (HMDBID = HMDB_ID, log2FC = foldchange_disorder, pvalue = p_values_disorder) #change column names



## Converting HMDB IDs to ChEBI IDs (BridgeDb)


In [ ]:
##Download the Metabolite mapping file (if it doesn't exist locally yet):
checkfile <- paste0(getwd(), '/' ,"data/metabolites.bridge")
if (!file.exists(checkfile)) {
download.file("https://figshare.com/ndownloader/files/26001794", checkfile)
}
#Load the ID mapper:
mapper <- loadDatabase(checkfile)

## Obtain the System codes for the databases HMDB (source database of dataset) and ChEBI (intended output database)
code_mappingFrom <- getSystemCode("HMDB")
code_mappingTo <- getSystemCode("ChEBI")

# TODO:: there are duplicated HMDB IDs in the datasets, maybe it should first be fixed based on the metabolite names
## Create a data frame with the mappings and the correct SystemCode
input <- data.frame(
    source = rep(code_mappingFrom, length(unique(c(mbx_dataset_CD$HMDBID, mbx_dataset_UC$HMDBID)))),
    identifier = unique(c(mbx_dataset_CD$HMDBID, mbx_dataset_UC$HMDBID)))
#Obtain all mappings from HMDB to ChEBI
chebiID <- maps(mapper, input, code_mappingTo) %>%
  filter(grepl("CHEBI", mapping, fixed = TRUE)) #remove all rows in the mapped data which do not include the prefix "CHEBI"
# checking the one-to-multiple mappings
if(!all(table(chebiID$identifier) == 1)) {print ("There are one-to-multiple mappings.")} else  print ("There is no one-to-multiple mapping.")

# add chebi IDs for each metabolite in the dataset
mbx_dataset_CD$ChEBI_BridgeDb <- chebiID$mapping[match(mbx_dataset_CD$HMDBID, chebiID$identifier)]
mbx_dataset_UC$ChEBI_BridgeDb <- chebiID$mapping[match(mbx_dataset_UC$HMDBID, chebiID$identifier)]


## Using BridgeDb for secondary to primary mapping of HMDB IDs

### mapping the HMDB IDs to primary HMDB IDs


In [ ]:
rm(list = setdiff(ls(), c("mbx_dataset_CD", "mbx_dataset_UC"))) # removing variables that are not required
##Download the mapping file (if it doesn't exist locally yet):
checkfile <- paste0(getwd(), '/' ,"data/hmdb_secondaryToPrimaryIDs.bridge")
if (!file.exists(checkfile)) {
download.file("NOT YET UPLOADED ANYWHERE", checkfile)
}

#Load the ID mapper:
mapper <- loadDatabase(checkfile)

## Obtain the System codes for the databases HMDB 
code_mapping <- getSystemCode("HMDB")

## Create a data frame with the mappings and the correct SystemCode
input <- data.frame(source= rep(code_mapping, length(unique(c(mbx_dataset_CD$HMDBID, mbx_dataset_UC$HMDBID)))),
                    identifier = gsub("\\*", "", unique(c(mbx_dataset_CD$HMDBID, mbx_dataset_UC$HMDBID))))

#converting secondary HMDB IDs to primary HMDB IDs 
hmdbID <- maps(mapper = mapper, input, target = code_mapping) %>% 
    filter(isPrimary == "T") # Keeping only rows where the mapping is annotated as primary id (defined in BridgeDb java library when creating the derby database)

# check the one-to-multiple mappings
if(!all(table(hmdbID$identifier) == 1)) {print ("There are one-to-multiple mappings.")} else  print ("There is no one-to-multiple mapping.")

# add HMDB id for each gene symbol in the dataset
mbx_dataset_CD$Current_HMDBID <- hmdbID$mapping[match(mbx_dataset_CD$HMDBID, hmdbID$identifier)]
mbx_dataset_UC$Current_HMDBID <- hmdbID$mapping[match(mbx_dataset_UC$HMDBID, hmdbID$identifier)]


Checking if all the secondary HMDB ids are mapped to a primary ID



In [ ]:
mbx_dataset_CD[is.na(mbx_dataset_CD$Current_HMDBID), ]



There is only one metabolite with no primary ID.
We try to finding a primary HMDB IDs for metabolites using the compound name


In [ ]:
# Get the metabolite name 
input <- c (mbx_dataset_CD$Compound_Name[is.na(mbx_dataset_CD$Current_HMDBID)], sub("(.)", "\\U\\1", mbx_dataset_CD$Compound_Name[is.na(mbx_dataset_CD$Current_HMDBID)], perl=TRUE)) #making sure that the metabolite would be mapped if in the database it starts with a capital letter

## Create a data frame with the mappings and the correct SystemCode
# When we created the derby database, the system code we used for the metabolite name was "O" as there was no defined system code for it
input <- data.frame(source = rep("O", length(input)), 
                    identifier = input)

#converting primary metabolite name to primary HMDB IDs 
(hmdbID <- maps(mapper = mapper, input, target = code_mapping) %>%
    filter(isPrimary == "T")) # Keeping only rows where the mapping is annotated as primary id (defined in BridgeDb java library when creating the derby database)


And the current id is HMDB0002172. Adding the current id to the mbx_dataset_CD



In [ ]:
mbx_dataset_CD$Current_HMDBID[mbx_dataset_CD$Compound_Name == "diacetylspermine"] = hmdbID$mapping
mbx_dataset_UC$Current_HMDBID[mbx_dataset_UC$Compound_Name == "diacetylspermine"] = hmdbID$mapping
rm(list = setdiff(ls(), c("mbx_dataset_CD", "mbx_dataset_UC"))) # removing variables that are not required



## Converting `primary` HMDB IDs to the corresponding ChEBI IDs (BridgeDb)


In [ ]:
rm(list = setdiff(ls(), c("mbx_dataset_CD", "mbx_dataset_UC"))) # removing variables that are not required
#Load the ID mapper:
mapper <- loadDatabase(paste0(getwd(), '/' ,"data/metabolites.bridge"))

## Obtain the System codes for the databases HMDB (source database of dataset) and ChEBI (intended output database)
code_mappingFrom <- getSystemCode("HMDB")
code_mappingTo <- getSystemCode("ChEBI")

# TODO:: there are duplicated HMDB IDs in the datasets, maybe it should first be fixed based on the metabolite names
## Create a data frame with the mappings and the correct SystemCode
input <- data.frame(
    source = rep(code_mappingFrom, length(unique(c(mbx_dataset_CD$Current_HMDBID, mbx_dataset_UC$Current_HMDBID)))),
    identifier = unique(c(mbx_dataset_CD$Current_HMDBID, mbx_dataset_UC$Current_HMDBID)))
#Obtain all mappings from primary HMDB ID to ChEBI ID
chebiID <- maps(mapper, input, code_mappingTo) %>%
  filter(grepl("CHEBI", mapping, fixed = TRUE)) #remove all rows in the mapped data which do not include the prefix "CHEBI"
# checking the one-to-multiple mappings
if(!all(table(chebiID$identifier) == 1)) {print ("There are one-to-multiple mappings.")} else  print ("There is no one-to-multiple mapping.")

# add chebi IDs for each metabolite in the dataset
mbx_dataset_CD$ChEBI_PriID_BridgeDb <- chebiID$mapping[match(mbx_dataset_CD$Current_HMDBID, chebiID$identifier)]
mbx_dataset_UC$ChEBI_PriID_BridgeDb <- chebiID$mapping[match(mbx_dataset_UC$Current_HMDBID, chebiID$identifier)]
rm(list = setdiff(ls(), c("mbx_dataset_CD", "mbx_dataset_UC"))) # removing variables that are not required



##Mapping stats:


In [ ]:
MappingStats <- data.table(stats =  c("#unique HMDB IDs in CD dataset\n(primary IDs for PrimaryID-nBridgeDb)",
                                     "#unique HMDB IDs in UC dataset\n(primary IDs for PrimaryID-nBridgeDb)",
                                     "#unique ChEBI IDs in CD dataset",
                                     "#unique ChEBI IDs in UC dataset",
                                     "#missing mappings for HMDB IDs to ChEBI IDs in CD dataset",
                                     "#missing mappings for HMDB IDs to ChEBI IDs in UC dataset"),
                           BridgeDb = c(length(unique(mbx_dataset_CD$HMDBID)),
                                        length(unique(mbx_dataset_UC$HMDBID)),
                                        length(na.omit(unique(mbx_dataset_CD$ChEBI_BridgeDb))),
                                        length(na.omit(unique(mbx_dataset_UC$ChEBI_BridgeDb))),
                                        sum(is.na(unique(mbx_dataset_CD %>% select(HMDBID, ChEBI_BridgeDb)) %>% .$ChEBI_BridgeDb)),
                                        sum(is.na(unique(mbx_dataset_UC %>% select(HMDBID, ChEBI_BridgeDb)) %>% .$ChEBI_BridgeDb))),
                           PrimaryID_BridgeDb = c(length(unique(mbx_dataset_CD$HMDBID)),
                                        length(unique(mbx_dataset_UC$HMDBID)),
                                        length(na.omit(unique(mbx_dataset_CD$ChEBI_PriID_BridgeDb))),
                                        length(na.omit(unique(mbx_dataset_UC$ChEBI_PriID_BridgeDb))),
                                        sum(is.na(unique(mbx_dataset_CD %>% select(HMDBID, ChEBI_PriID_BridgeDb)) %>% .$ChEBI_PriID_BridgeDb)),
                                        sum(is.na(unique(mbx_dataset_UC %>% select(HMDBID, ChEBI_PriID_BridgeDb)) %>% .$ChEBI_PriID_BridgeDb))))

kable(MappingStats)

#ID mapping visualization
order <- c("#unique HMDB IDs in CD dataset\n(primary IDs for PrimaryID-nBridgeDb)" = 1,
           "#unique ChEBI IDs in CD dataset" = 2,
           "#missing mappings for HMDB IDs to ChEBI IDs in CD dataset" = 3, 
           "#unique HMDB IDs in UC dataset\n(primary IDs for PrimaryID-nBridgeDb)" = 4,
           "#unique ChEBI IDs in UC dataset" = 5, 
           "#missing mappings for HMDB IDs to ChEBI IDs in UC dataset" = 6)

MappingStats_toPlot <- MappingStats %>%
  reshape2::melt() %>%
  mutate(order=order[match(stats,names(order))],
         rescale = scales::rescale(value))


(plot <- ggplot(MappingStats_toPlot, aes(x = variable, y = reorder(stats, -order))) +
      geom_tile(aes(fill = rescale), colour = "white",
                lwd = 1.5,
              linetype = 1) +
    coord_fixed() +
    scale_fill_gradient(low = "white", high = "orange") +
    theme_bw() +
    labs(y = NULL, x = NULL) +
    geom_text(aes(label = value), color = "black", size = 1) +
    scale_x_discrete(position = "top", 
                     labels = c("BridgeDb", "PrimaryID\nBridgeDb")) + 
    theme(axis.text.x.top = element_text(vjust = 0.5, size = 4, face = "bold"),
          axis.text.y = element_text(size = 5, hjust = 0),
          legend.position = "none"))

# ggsave(plot, device = "jpeg", units = "cm", width = 7, height = 6, filename = "figures/MappingStats_metabolomics.jpeg")



##Save data, print session info, and citation


In [ ]:
##Save data: exporting results to the file
#CD
write.table(mbx_dataset_CD, file = "results/mbx_IDMapping_CD", 
            sep = "\t" , quote = FALSE, row.names = FALSE)

#UC
write.table(mbx_dataset_UC, file = "results/mbx_IDMapping_UC", 
            sep = "\t" , quote = FALSE, row.names = FALSE)

##Print session info:
sessionInfo()

##Citation BridgeDb:
citation("BridgeDbR")

##Remove add the data objects
rm (list = ls())


In [ ]:
#Jupyter Notebook file
# if(!"devtools" %in% installed.packages()) BiocManager::install("devtools")
# if(!"rmd2jupyter" %in% installed.packages()) devtools::install_github("mkearney/rmd2jupyter", force=TRUE)
# library(devtools)
# library(rmd2jupyter)
# setwd(dirname(rstudioapi::getSourceEditorContext()$path))
# rmd2jupyter("identifier_mapping_metabolomics.Rmd")
